In [1]:
# ============================================================
# FLIGHT DISRUPTION INFERENCE
# OvR MODELS + SOFTMAX + ARGMAX
# USER INPUT | MEMORY SAFE
# ============================================================

import pandas as pd
import numpy as np
import joblib
import gc
from scipy.special import softmax

# ============================================================
# 1. FEATURE LIST (MUST MATCH TRAINING)
# ============================================================

FEATURES = [
    'DayOfWeek',
    'DayofMonth',
    'Month',
    'Distance',
    'CRSDepMin',
    'CRSArrMin',
    'ScheduledElapsedTime',
    'OriginReliability',
    'DestReliability',
    'CarrierReliability',
    'DepTimeOfDay_enc',
    'ArrTimeOfDay_enc'
]

# ============================================================
# 2. LOAD ENCODINGS
# ============================================================

carrier_df = pd.read_csv(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\carrier_reliability_encoding.csv"
)
origin_df = pd.read_csv(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\origin_reliability_encoding.csv"
)
dest_df = pd.read_csv(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\dest_reliability_encoding.csv"
)

carrier_map = dict(zip(carrier_df["UniqueCarrier"], carrier_df["CarrierReliability"]))
origin_map  = dict(zip(origin_df["Origin"], origin_df["OriginReliability"]))
dest_map    = dict(zip(dest_df["Dest"], dest_df["DestReliability"]))

# ============================================================
# 3. TIME OF DAY ENCODING (SAME AS TRAINING)
# ============================================================

def time_of_day(hour):
    if 5 <= hour < 11:
        return 0   # Morning
    elif 11 <= hour < 17:
        return 1   # Afternoon
    elif 17 <= hour < 22:
        return 2   # Evening
    else:
        return 3   # Night

# ============================================================
# 4. USER INPUT
# ============================================================

print("\n✈️ ENTER FLIGHT DETAILS")

carrier = input("Carrier (AA, DL, UA): ").upper().strip()
origin  = input("Origin Airport (JFK): ").upper().strip()
dest    = input("Destination Airport (LAX): ").upper().strip()

dep_hour = int(input("Departure hour (0–23): "))
arr_hour = int(input("Arrival hour (0–23): "))

# ============================================================
# 5. VALIDATION
# ============================================================

if carrier not in carrier_map:
    raise ValueError(f"Unknown carrier: {carrier}")
if origin not in origin_map:
    raise ValueError(f"Unknown origin airport: {origin}")
if dest not in dest_map:
    raise ValueError(f"Unknown destination airport: {dest}")

# ============================================================
# 6. PREPROCESS INPUT (STRICT FEATURE ALIGNMENT)
# ============================================================

row = {
    "CarrierReliability": carrier_map[carrier],
    "OriginReliability": origin_map[origin],
    "DestReliability": dest_map[dest],
    "DepTimeOfDay_enc": time_of_day(dep_hour),
    "ArrTimeOfDay_enc": time_of_day(arr_hour),
}

# Fill missing numeric features with 0 (same as training default)
for f in FEATURES:
    row.setdefault(f, 0)

X = pd.DataFrame([row])[FEATURES]

# ============================================================
# 7. OvR MODELS (LOAD → SCORE → FREE)
# ============================================================

def load_score_free(path, X):
    model = joblib.load(path)
    score = model.predict_proba(X)[0, 1]
    del model
    gc.collect()
    return score

scores = {}

scores["Cancelled"] = load_score_free(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\OneVRest_models\\ovr_extratrees_Cancelled.joblib",
    X
)

scores["Diverted"] = load_score_free(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\OneVRest_models\\ovr_extratrees_Diverted.joblib",
    X
)

scores["Delayed"] = load_score_free(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\OneVRest_models\\ovr_extratrees_Delayed.joblib",
    X
)

scores["On Time"] = load_score_free(
    "C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\OneVRest_models\\ovr_extratrees_On Time.joblib",
    X
)

# ============================================================
# 8. SOFTMAX COUPLING (TRUE PROBABILITIES)
# ============================================================

CLASS_ORDER = ["Cancelled", "Diverted", "Delayed", "On Time"]

score_vec = np.array([scores[c] for c in CLASS_ORDER])
probs = softmax(score_vec)

# ============================================================
# 9. ARGMAX DECISION
# ============================================================

pred_idx = np.argmax(probs)
final_prediction = CLASS_ORDER[pred_idx]

# ============================================================
# 10. RISK SCORE
# ============================================================

risk_score = 1 - probs[CLASS_ORDER.index("On Time")]

# ============================================================
# 11. OUTPUT
# ============================================================

print("\n🧠 FINAL INFERENCE RESULT")
print("--------------------------------------------------")
print(f"Carrier     : {carrier}")
print(f"Route       : {origin} → {dest}")
print(f"Departure   : {dep_hour}:00")
print(f"Arrival     : {arr_hour}:00")

print("\nClass Probabilities:")
for cls, p in zip(CLASS_ORDER, probs):
    print(f"{cls:10s}: {p:.3f}")

print(f"\n📌 FINAL PREDICTION : {final_prediction}")
print(f"⚠️  RISK SCORE     : {risk_score:.3f}")


✈️ ENTER FLIGHT DETAILS

🧠 FINAL INFERENCE RESULT
--------------------------------------------------
Carrier     : AA
Route       : JFK → LAX
Departure   : 13:00
Arrival     : 15:00

Class Probabilities:
Cancelled : 0.255
Diverted  : 0.219
Delayed   : 0.266
On Time   : 0.260

📌 FINAL PREDICTION : Delayed
⚠️  RISK SCORE     : 0.740
